<a href="https://colab.research.google.com/github/Himanshu-Tagde/ABC/blob/main/streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
pip install streamlit

In [16]:
!unzip /content/images.zip

Archive:  /content/images.zip
   creating: images/
  inflating: images/1.png            
  inflating: images/10.png           
  inflating: images/11.png           
  inflating: images/12.png           
  inflating: images/13.png           
  inflating: images/14.png           
  inflating: images/15.png           
  inflating: images/16.png           
  inflating: images/17.png           
  inflating: images/18.png           
  inflating: images/19.png           
  inflating: images/2.png            
  inflating: images/20.png           
  inflating: images/3.png            
  inflating: images/4.png            
  inflating: images/5.png            
  inflating: images/6.png            
  inflating: images/7.png            
  inflating: images/8.png            
  inflating: images/9.png            
  inflating: images/divide.png       
  inflating: images/minus.png        
  inflating: images/multiply.png     
  inflating: images/plus.png         


In [48]:
#update 3.3
%%writefile app.py
import streamlit as st
import random
from PIL import Image

# Title of the app
st.title("Kids Learning App: Math Fun with Pictures!")

# Initialize score and other session state variables
if "score" not in st.session_state:
    st.session_state.score = 0

if "question_generated" not in st.session_state:
    st.session_state.question_generated = False

if "selected_answer" not in st.session_state:
    st.session_state.selected_answer = None

if "feedback" not in st.session_state:
    st.session_state.feedback = ""

if "correct_answer" not in st.session_state:
    st.session_state.correct_answer = None

if "submitted" not in st.session_state:
    st.session_state.submitted = False

# Function to reset the game
def reset_game():
    st.session_state.selected_answer = None
    st.session_state.feedback = ""
    st.session_state.submitted = False
    st.session_state.question_generated = False

# Function to quit the game
def quit_game():
    st.session_state.score = 0
    st.session_state.question_generated = False
    st.write("Thanks for playing! The game has ended. Goodbye!")

# Display the current score
st.write(f"Your current score: {st.session_state.score}")

# Display a welcome message
st.write("Welcome to the math learning game! Let's have fun while learning math.")
st.write("Choose the operation you want to practice:")

# Dropdown menu to select math operation
operation = st.selectbox("Choose a math operation", ("Addition", "Subtraction", "Multiplication", "Division"))

# Function to generate a math question with images instead of numbers
def generate_question(operation):
    if operation == "Addition":
        a = random.randint(1, 20)
        b = random.randint(1, 20)
        answer = a + b
        question = f"What is {a} + {b}?"
        sign_image = "plus.png"
    elif operation == "Subtraction":
        a = random.randint(1, 20)
        b = random.randint(1, a) # ensure no negative result
        answer = a - b
        question = f"What is {a} - {b}?"
        sign_image = "minus.png"
    elif operation == "Multiplication":
        a = random.randint(1, 10)
        b = random.randint(1, 10)
        answer = a * b
        question = f"What is {a} x {b}?"
        sign_image = "multiply.png"
    elif operation == "Division":
        a = random.randint(2, 20)
        b = random.randint(2, 10)
        answer = a // b # Integer division for simplicity
        question = f"What is {a} ÷ {b}?"
        sign_image = "divide.png"
    return question, answer, a, b, sign_image

# Function to display the number images with error handling
def display_number_image(num):
    try:
        img = Image.open(f"images/{num}.png")
        st.image(img, caption=str(num), width=100)
    except FileNotFoundError:
        st.write(f"Image for number {num} not found. Please ensure the image is in the 'images' folder.")

# Function to display the operation sign image with error handling
def display_sign_image(sign_image):
    try:
        img = Image.open(f"images/{sign_image}")
        st.image(img, width=50)
    except FileNotFoundError:
        st.write(f"Image for operation sign {sign_image} not found. Please ensure the image is in the 'images' folder.")

# Reload button logic
if st.button("Reload Game"):
    reset_game()
    st.write("The game has been reset. Let's start again!")

# Quit button logic
if st.button("Quit Game"):
    quit_game()
    st.stop() # This stops further Streamlit code from executing, effectively quitting the app

# Generate a random math question based on selected operation
if not st.session_state.get("question_generated", False):
    question, correct_answer, a, b, sign_image = generate_question(operation)
    st.session_state.question_generated = True
    st.session_state.correct_answer = correct_answer

    # Display the question with images
    st.write("What is this?")
    col1, col2, col3 = st.columns(3)

    with col1:
        display_number_image(a)

    with col2:
        display_sign_image(sign_image) # Display the operation sign image

    with col3:
        display_number_image(b)

    st.write(question)

    # Generate multiple-choice options
    options = [correct_answer]
    while len(options) < 4:
        random_option = random.randint(correct_answer - 5, correct_answer + 5)
        if random_option not in options and random_option >= 0: # avoid duplicates and negative numbers
            options.append(random_option)

    random.shuffle(options) # Shuffle the options

    # Display the options as radio buttons
    selected_option = st.radio("Choose the correct answer:", options, key="answer_options")

    # Save the selected answer in session state
    st.session_state.selected_answer = selected_option

# Submit button logic
if st.button("Submit"):
    if st.session_state.selected_answer is not None:
        # Check if the answer is correct
        if st.session_state.selected_answer == st.session_state.correct_answer:
            st.session_state.feedback = "Great job! You got it right."
            st.session_state.score += 1 # Increment the score when the answer is correct
        else:
            st.session_state.feedback = f"Oops! The correct answer was {st.session_state.correct_answer}. Try again!"
    else:
        st.session_state.feedback = "Please select an answer before submitting."

    st.write(st.session_state.feedback)
    st.session_state.submitted = True

# Display updated score after each question
st.write(f"Your current score: {st.session_state.score}")

# Option to generate a new question
if st.button("Next Question"):
    reset_game() # Reset the game to generate a new question
    st.session_state.question_generated = False
    st.session_state.submitted = False # Reset submission state
    st.write("Get ready for the next question!")

Writing app.py


In [49]:
!wget -q -O - ipv4.icanhazip.com

35.221.157.251


In [50]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.221.157.251:8501

⠼⠴your url is: https://seven-flies-trade.loca.lt
  Stopping...
^C
